<a href="https://colab.research.google.com/github/xrueiii/2025IMProject/blob/main/%E5%B0%88%E9%A1%8C_%E5%88%A4%E6%96%B7%E9%87%8D%E8%A4%87%E6%96%87%E7%AB%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pandas sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

def preprocess_text(text):
    """基本文本預處理：去除特殊字符，轉換為小寫"""
    if pd.isna(text):
        return ""
    # 去除引號、換行符等
    text = re.sub(r'[\"\'\n\r\\]', ' ', str(text))
    # 轉換為小寫
    return text.lower()

def find_similar_articles(file_path, similarity_threshold=0.8, columns_to_check=None, output_file=None):
    """
    找出CSV文件中相似度高的文章對

    參數:
    file_path: CSV文件路徑
    similarity_threshold: 相似度閾值（0-1之間，越高表示越相似）
    columns_to_check: 要檢查相似度的列名列表，None表示檢查文章內容相關的列
    output_file: 輸出結果的文件路徑

    返回:
    相似文章對的DataFrame
    """
    print(f"正在讀取CSV文件: {file_path}")
    # 讀取CSV文件
    try:
        df = pd.read_csv(file_path)
        print(f"成功讀取CSV文件，共有{len(df)}行")
        print(f"CSV文件的列名: {list(df.columns)}")
    except Exception as e:
        print(f"讀取文件出錯: {e}")
        return None

    # 確定要檢查的列
    if columns_to_check is None:
        # 嘗試自動識別可能包含文章內容的列
        possible_content_columns = ['ARTICLE_TEXT', 'article_text', 'text', 'content', 'body', 'full_text']
        columns_to_check = []
        for col in possible_content_columns:
            if col in df.columns:
                columns_to_check.append(col)
                break

        # 如果找不到明顯的內容列，檢查是否有可能包含文章內容的列
        if not columns_to_check:
            for col in df.columns:
                if 'text' in col.lower() or 'content' in col.lower() or 'article' in col.lower():
                    columns_to_check.append(col)
                    break

        # 如果還是找不到，使用最後一列（假設文章內容通常是最後一列）
        if not columns_to_check and len(df.columns) > 0:
            text_col = df.columns[-1]
            # 檢查最後一列的內容是否看起來像文章（平均長度超過100個字符）
            avg_length = df[text_col].astype(str).str.len().mean()
            if avg_length > 100:
                columns_to_check = [text_col]
                print(f"沒有找到明確的文章內容列，將使用最後一列 '{text_col}' 作為內容列（平均長度: {avg_length:.1f}個字符）")
            else:
                # 嘗試找出平均長度最長的列
                lengths = {col: df[col].astype(str).str.len().mean() for col in df.columns}
                longest_col = max(lengths.items(), key=lambda x: x[1])
                if longest_col[1] > 100:
                    columns_to_check = [longest_col[0]]
                    print(f"將使用平均長度最長的列 '{longest_col[0]}' 作為內容列（平均長度: {longest_col[1]:.1f}個字符）")
                else:
                    print("無法找到適合的文章內容列，請手動指定columns_to_check參數")
                    return None
    else:
        # 確保所有指定的列都存在
        missing_cols = [col for col in columns_to_check if col not in df.columns]
        if missing_cols:
            print(f"警告：以下指定的列不存在於CSV中: {missing_cols}")
            columns_to_check = [col for col in columns_to_check if col in df.columns]

        if not columns_to_check:
            print("所有指定的列都不存在於CSV中，無法進行分析")
            return None

    print(f"將檢查以下列的相似度: {columns_to_check}")

    # 準備要分析的文本
    combined_texts = []
    for _, row in df.iterrows():
        combined_text = " ".join([preprocess_text(row[col]) for col in columns_to_check])
        combined_texts.append(combined_text)

    # 使用TF-IDF向量化文本
    print("正在向量化文本...")
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform(combined_texts)
    except Exception as e:
        print(f"向量化文本出錯: {e}")
        return None

    # 計算餘弦相似度
    print("正在計算文本相似度...")
    cosine_sim = cosine_similarity(tfidf_matrix)

    # 找出高相似度的文章對
    print(f"正在尋找相似度高於 {similarity_threshold} 的文章對...")
    similar_pairs = []

    for i in range(len(df)):
        # 只檢查i之後的文章，避免重複比較和自身比較
        for j in range(i+1, len(df)):
            similarity = cosine_sim[i, j]
            if similarity >= similarity_threshold:
                pair_info = {
                    'similarity': similarity
                }

                # 添加兩篇文章的識別信息
                for article_idx, prefix in [(i, 'article1_'), (j, 'article2_')]:
                    # 檢查是否有id列
                    if 'id' in df.columns:
                        pair_info[f'{prefix}id'] = df.iloc[article_idx]['id']
                    else:
                        pair_info[f'{prefix}id'] = article_idx

                    # 檢查是否有title列
                    if 'title' in df.columns:
                        pair_info[f'{prefix}title'] = df.iloc[article_idx]['title']

                    # 添加一小部分內容作為預覽
                    for col in columns_to_check:
                        preview_text = str(df.iloc[article_idx][col])
                        if len(preview_text) > 100:
                            preview_text = preview_text[:100] + "..."
                        pair_info[f'{prefix}{col}_preview'] = preview_text

                similar_pairs.append(pair_info)

    # 轉換為DataFrame並按相似度降序排序
    if similar_pairs:
        result_df = pd.DataFrame(similar_pairs)
        result_df = result_df.sort_values(by='similarity', ascending=False)

        # 輸出結果
        print(f"找到 {len(result_df)} 對相似文章")
        if output_file:
            result_df.to_csv(output_file, index=False)
            print(f"結果已保存到 {output_file}")

        return result_df
    else:
        print("未找到符合閾值的相似文章對")
        return pd.DataFrame()

# 使用示例
if __name__ == "__main__":
    # 替換為您的CSV文件路徑
    file_path = "articles.csv"

    # 自動識別合適的列 - 您也可以明確指定要檢查的列
    # columns_to_check = ['ARTICLE_TEXT']

    # 查找相似度大於0.8的文章
    similar_articles = find_similar_articles(
        file_path=file_path,
        similarity_threshold=0.8,
        # columns_to_check=columns_to_check,  # 不指定，讓程序自動識別
        output_file="similar_articles.csv"
    )

    # 顯示結果的前幾行
    if similar_articles is not None and not similar_articles.empty:
        print("\n相似文章對的前5行:")
        print(similar_articles.head())

正在讀取CSV文件: articles.csv
成功讀取CSV文件，共有634行
CSV文件的列名: ['id', 'title', 'author', 'publisher', 'date', 'summary', 'ARTICLE_TEXT', 'media_type', 'status']
將檢查以下列的相似度: ['ARTICLE_TEXT']
正在向量化文本...
正在計算文本相似度...
正在尋找相似度高於 0.8 的文章對...
找到 62 對相似文章
結果已保存到 similar_articles.csv

相似文章對的前5行:
    similarity  article1_id  \
3          1.0           23   
45         1.0          431   
40         1.0          362   
44         1.0          408   
2          1.0            9   

                                       article1_title  \
3      Anti-Asian attacks must no longer be minimized   
45  Epidemic of hate as attacks on Asian Americans...   
40  'I feel like it's more blatant': Students afra...   
44  Biases you didn't know existed in the healthca...   
2   National group received 67 reports of anti-Asi...   

                        article1_ARTICLE_TEXT_preview  article2_id  \
3   After Dylan Adler was screamed at, chased and ...           29   
45  Four prominent Asian Americans says prejudice,... 

In [ ]:
import pandas as pd

def remove_duplicate_articles(articles_path, similarity_path, threshold=0.89, date_col='date', id_col='id'):
    # 讀取資料
    articles_df = pd.read_csv("/content/articles.csv")
    sim_df = pd.read_csv("/content/similar_articles.csv")

    # 儲存要刪除的 ID
    ids_to_remove = set()

    for _, row in sim_df.iterrows():
        if row['similarity'] > threshold:
            id1, id2 = int(row['article1_id']), int(row['article2_id'])

            # 根據日期判斷誰是比較新的
            date1 = pd.to_datetime(articles_df.loc[articles_df[id_col] == id1, date_col].values[0])
            date2 = pd.to_datetime(articles_df.loc[articles_df[id_col] == id2, date_col].values[0])

            if date1 > date2:
                ids_to_remove.add(id1)
            else:
                ids_to_remove.add(id2)

    # 刪除重複文章
    before_count = len(articles_df)
    filtered_df = articles_df[~articles_df[id_col].isin(ids_to_remove)]
    after_count = len(filtered_df)

    print(f"共刪除了 {before_count - after_count} 篇文章")
    print(f"剩餘 {after_count} 篇文章")
    print("被刪除的文章 ID：", sorted(ids_to_remove))

    return filtered_df

# 使用範例
filtered_articles = remove_duplicate_articles(
    articles_path="articles.csv",
    similarity_path="similarities.csv",
    threshold=0.89
)

# 可選：儲存結果
filtered_articles.to_csv("articles_deduplicated.csv", index=False)


共刪除了 47 篇文章
剩餘 587 篇文章
被刪除的文章 ID： [7, 29, 41, 51, 54, 55, 63, 64, 77, 78, 95, 106, 111, 135, 138, 146, 215, 229, 234, 254, 261, 271, 287, 289, 295, 353, 354, 362, 372, 378, 407, 408, 432, 444, 447, 462, 464, 472, 482, 491, 497, 498, 502, 509, 608, 612, 615]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

def preprocess_text(text):
    """基本文本預處理：去除特殊字符，轉換為小寫"""
    if pd.isna(text):
        return ""
    # 去除引號、換行符等
    text = re.sub(r'[\"\'\n\r\\]', ' ', str(text))
    # 轉換為小寫
    return text.lower()

def find_similar_articles(file_path, similarity_threshold=0.8, columns_to_check=None, output_file=None):
    """
    找出CSV文件中相似度高的文章對

    參數:
    file_path: CSV文件路徑
    similarity_threshold: 相似度閾值（0-1之間，越高表示越相似）
    columns_to_check: 要檢查相似度的列名列表，None表示檢查文章內容相關的列
    output_file: 輸出結果的文件路徑

    返回:
    相似文章對的DataFrame
    """
    print(f"正在讀取CSV文件: {file_path}")
    # 讀取CSV文件
    try:
        df = pd.read_csv(file_path)
        print(f"成功讀取CSV文件，共有{len(df)}行")
        print(f"CSV文件的列名: {list(df.columns)}")
    except Exception as e:
        print(f"讀取文件出錯: {e}")
        return None

    # 確定要檢查的列
    if columns_to_check is None:
        # 嘗試自動識別可能包含文章內容的列
        possible_content_columns = ['ARTICLE_TEXT', 'article_text', 'text', 'content', 'body', 'full_text']
        columns_to_check = []
        for col in possible_content_columns:
            if col in df.columns:
                columns_to_check.append(col)
                break

        # 如果找不到明顯的內容列，檢查是否有可能包含文章內容的列
        if not columns_to_check:
            for col in df.columns:
                if 'text' in col.lower() or 'content' in col.lower() or 'article' in col.lower():
                    columns_to_check.append(col)
                    break

        # 如果還是找不到，使用最後一列（假設文章內容通常是最後一列）
        if not columns_to_check and len(df.columns) > 0:
            text_col = df.columns[-1]
            # 檢查最後一列的內容是否看起來像文章（平均長度超過100個字符）
            avg_length = df[text_col].astype(str).str.len().mean()
            if avg_length > 100:
                columns_to_check = [text_col]
                print(f"沒有找到明確的文章內容列，將使用最後一列 '{text_col}' 作為內容列（平均長度: {avg_length:.1f}個字符）")
            else:
                # 嘗試找出平均長度最長的列
                lengths = {col: df[col].astype(str).str.len().mean() for col in df.columns}
                longest_col = max(lengths.items(), key=lambda x: x[1])
                if longest_col[1] > 100:
                    columns_to_check = [longest_col[0]]
                    print(f"將使用平均長度最長的列 '{longest_col[0]}' 作為內容列（平均長度: {longest_col[1]:.1f}個字符）")
                else:
                    print("無法找到適合的文章內容列，請手動指定columns_to_check參數")
                    return None
    else:
        # 確保所有指定的列都存在
        missing_cols = [col for col in columns_to_check if col not in df.columns]
        if missing_cols:
            print(f"警告：以下指定的列不存在於CSV中: {missing_cols}")
            columns_to_check = [col for col in columns_to_check if col in df.columns]

        if not columns_to_check:
            print("所有指定的列都不存在於CSV中，無法進行分析")
            return None

    print(f"將檢查以下列的相似度: {columns_to_check}")

    # 準備要分析的文本
    combined_texts = []
    for _, row in df.iterrows():
        combined_text = " ".join([preprocess_text(row[col]) for col in columns_to_check])
        combined_texts.append(combined_text)

    # 使用TF-IDF向量化文本
    print("正在向量化文本...")
    vectorizer = TfidfVectorizer(stop_words='english')
    try:
        tfidf_matrix = vectorizer.fit_transform(combined_texts)
    except Exception as e:
        print(f"向量化文本出錯: {e}")
        return None

    # 計算餘弦相似度
    print("正在計算文本相似度...")
    cosine_sim = cosine_similarity(tfidf_matrix)

    # 找出高相似度的文章對
    print(f"正在尋找相似度高於 {similarity_threshold} 的文章對...")
    similar_pairs = []

    for i in range(len(df)):
        # 只檢查i之後的文章，避免重複比較和自身比較
        for j in range(i+1, len(df)):
            similarity = cosine_sim[i, j]
            if similarity >= similarity_threshold:
                pair_info = {
                    'similarity': similarity
                }

                # 添加兩篇文章的識別信息
                for article_idx, prefix in [(i, 'article1_'), (j, 'article2_')]:
                    # 檢查是否有id列
                    if 'id' in df.columns:
                        pair_info[f'{prefix}id'] = df.iloc[article_idx]['id']
                    else:
                        pair_info[f'{prefix}id'] = article_idx

                    # 檢查是否有title列
                    if 'title' in df.columns:
                        pair_info[f'{prefix}title'] = df.iloc[article_idx]['title']

                    # 添加一小部分內容作為預覽
                    for col in columns_to_check:
                        preview_text = str(df.iloc[article_idx][col])
                        if len(preview_text) > 100:
                            preview_text = preview_text[:100] + "..."
                        pair_info[f'{prefix}{col}_preview'] = preview_text

                similar_pairs.append(pair_info)

    # 轉換為DataFrame並按相似度降序排序
    if similar_pairs:
        result_df = pd.DataFrame(similar_pairs)
        result_df = result_df.sort_values(by='similarity', ascending=False)

        # 輸出結果
        print(f"找到 {len(result_df)} 對相似文章")
        if output_file:
            result_df.to_csv(output_file, index=False)
            print(f"結果已保存到 {output_file}")

        return result_df
    else:
        print("未找到符合閾值的相似文章對")
        return pd.DataFrame()

# 使用示例
if __name__ == "__main__":
    # 替換為您的CSV文件路徑
    file_path = "/content/articles_deduplicated.csv"

    # 自動識別合適的列 - 您也可以明確指定要檢查的列
    # columns_to_check = ['ARTICLE_TEXT']

    # 查找相似度大於0.8的文章
    similar_articles = find_similar_articles(
        file_path=file_path,
        similarity_threshold=0.8,
        # columns_to_check=columns_to_check,  # 不指定，讓程序自動識別
        output_file="similar_articles_round2.csv"
    )

    # 顯示結果的前幾行
    if similar_articles is not None and not similar_articles.empty:
        print("\n相似文章對的前5行:")
        print(similar_articles.head())

正在讀取CSV文件: /content/articles_deduplicated.csv
成功讀取CSV文件，共有587行
CSV文件的列名: ['id', 'title', 'author', 'publisher', 'date', 'summary', 'ARTICLE_TEXT', 'media_type', 'status']
將檢查以下列的相似度: ['ARTICLE_TEXT']
正在向量化文本...
正在計算文本相似度...
正在尋找相似度高於 0.8 的文章對...
找到 6 對相似文章
結果已保存到 similar_articles_round2.csv

相似文章對的前5行:
   similarity  article1_id                                     article1_title  \
1    0.887819           73  How Anti-Asian Activity Online Set the Stage f...   
5    0.885127          590  Virus ravages San Francisco's Asian American c...   
2    0.849716           81  A Tense Lunar New Year for the Bay Area After ...   
4    0.848496          590  Virus ravages San Francisco's Asian American c...   
3    0.840703          269  CANADIANS MUST ‘STAND UP' IN FIGHT AGAINST ANT...   

                       article1_ARTICLE_TEXT_preview  article2_id  \
1  Protesters gather for a silent vigil in the Ch...          250   
5  SAN FRANCISCO – Mandy Rong was terrified her 1...          595   
2 

根據 round_2 的similarity 計算之後，在下面的程式刪除了 [590, 595, 269] 這三篇文章，因此目前共刪除了 50 篇文章，剩餘 584 篇文章

In [4]:
import pandas as pd

# 讀取文章
articles_df = pd.read_csv("/content/articles_deduplicated.csv")

# 1. 刪除特定 ID
ids_to_delete = [590, 595, 269]
filtered_df = articles_df[~articles_df["id"].isin(ids_to_delete)]

# 轉換日期欄位為 datetime 格式
filtered_df["date"] = pd.to_datetime(filtered_df["date"], format="%Y/%m/%d", errors='coerce')

# 2. 統計每個出版社總共有幾篇文章
publisher_total_counts = filtered_df["publisher"].value_counts().reset_index()
publisher_total_counts.columns = ["publisher", "total_articles"]

# 3. 統計每個出版社在 2021/3/16 之前的文章數
cutoff_date = pd.to_datetime("2021/03/16")
filtered_before_cutoff = filtered_df[filtered_df["date"] < cutoff_date]
publisher_before_counts = filtered_before_cutoff["publisher"].value_counts().reset_index()
publisher_before_counts.columns = ["publisher", "articles_before_2021_03_16"]

# 4. 合併兩個統計表格
publisher_stats = pd.merge(
    publisher_total_counts,
    publisher_before_counts,
    on="publisher",
    how="left"
)

# 將 NaN 補 0 並轉成整數（某些出版社可能沒在 cutoff 前發表）
publisher_stats["articles_before_2021_03_16"] = publisher_stats["articles_before_2021_03_16"].fillna(0).astype(int)

# 儲存處理後的文章與統計結果
filtered_df.to_csv("articles_after_manual_delete.csv", index=False)
publisher_stats.to_csv("publisher_article_stats.csv", index=False)

# 顯示統計結果
print(publisher_stats)


<ipython-input-4-464968845cd2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["date"] = pd.to_datetime(filtered_df["date"], format="%Y/%m/%d", errors='coerce')


         publisher  total_articles  articles_before_2021_03_16
0           Others             429                           0
1        USA Today              45                           0
2         LA Times              31                           0
3   New York Times              27                           0
4  Washington Post              24                           0
5     Boston Globe              17                           0
6  Chicago Tribune               8                           0
7     Star Tribune               3                           0


In [5]:
import pandas as pd

# 讀入文章資料
articles_df = pd.read_csv("/content/articles_deduplicated.csv")

# 1. 刪除指定的文章 ID
ids_to_delete = [590, 595, 269]
filtered_df = articles_df[~articles_df["id"].isin(ids_to_delete)]

# 2. 轉換 date 為 datetime 格式（格式為 yyyy-mm-dd）
filtered_df["date"] = pd.to_datetime(filtered_df["date"], format="%Y-%m-%d", errors='coerce')

# 3. 設定事件日期（2021/3/16）
event_date = pd.to_datetime("2021-03-16")

# 4. 計算總數、事件前、事件後文章數（以 publisher 分組）
grouped = filtered_df.groupby("publisher")

stats = pd.DataFrame({
    "total_articles": grouped.size(),
    "articles_before_event": grouped.apply(lambda x: (x["date"] < event_date).sum()),
    "articles_after_event": grouped.apply(lambda x: (x["date"] >= event_date).sum())
}).reset_index()

# 5. 儲存結果
filtered_df.to_csv("articles_after_manual_delete.csv", index=False)
stats.to_csv("publisher_article_stats.csv", index=False)

# 顯示結果
print(stats)


         publisher  total_articles  articles_before_event  \
0     Boston Globe              17                      4   
1  Chicago Tribune               8                      4   
2         LA Times              31                     11   
3   New York Times              27                     16   
4           Others             429                    217   
5     Star Tribune               3                      0   
6        USA Today              45                     19   
7  Washington Post              24                     15   

   articles_after_event  
0                    13  
1                     4  
2                    20  
3                    11  
4                   212  
5                     3  
6                    26  
7                     9  


<ipython-input-5-bc32e14bb68d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["date"] = pd.to_datetime(filtered_df["date"], format="%Y-%m-%d", errors='coerce')
<ipython-input-5-bc32e14bb68d>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  "articles_before_event": grouped.apply(lambda x: (x["date"] < event_date).sum()),
<ipython-input-5-bc32e14bb68d>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavi